In [2]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

In [3]:
%%px 

import numpy as np
import io
import urllib
import requests
from mpi4py import MPI


In [4]:
%%px

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))
    
    
N = 1024
nx = N//8
stride = nx*N*N
farr = np.zeros((nx*N*N,),dtype='float32')

[stdout:1] 
n_proc = 8
rank = 0


In [5]:
%%px

# simple test function that retrieves a complete file and returns its size
# can switch between urllib and requests library
def get_url(url,use_requests=False):
    data=urllib.request.urlopen(url).read()
    return data

In [16]:
%%px

comm.Barrier(); t1=MPI.Wtime()

virgo_url = 'http://virgo02/turbdb/turbdb10'+str(rank+1)+'_0.bin' 
d=urllib.request.urlopen(virgo_url).read()
n=int(len(d)/4)
_arr=np.frombuffer(d,dtype=np.float32,count=n)

d=None

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    print("Finished loading")
    print('Load field in memory from virgo: {0:.2f} seconds\n'.format(t2-t1))
    print("shape == 3*(512**3): ",_arr.shape[0] == 3*(512**3))

[stdout:1] 
Finished loading
Load field in memory from virgo: 26.07 seconds

shape == 3*(512**3):  True


In [17]:
%%px

comm.Barrier(); t1=MPI.Wtime()

virgo_url = 'http://virgo02/turbulence/part'+str(rank+1)+'/turbdb10'+str(rank+1)+'_0.bin'
d=urllib.request.urlopen(virgo_url).read()
n=int(len(d)/4)
_arr=np.frombuffer(d,dtype=np.float32,count=n)

d=None

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    print("Finished loading")
    print('Load field in memory from virgo: {0:.2f} seconds\n'.format(t2-t1))
    print("shape == 3*(512**3): ",_arr.shape[0] == 3*(512**3))

[stdout:1] 
Finished loading
Load field in memory from virgo: 26.71 seconds

shape == 3*(512**3):  True


In [18]:
%%px

comm.Barrier(); t1=MPI.Wtime()

virgo_url = 'http://dsp060/disk0'+str(rank+1)+'/filedb/turbulence/turbdb10'+str(rank+1)+'_0.bin'
d=urllib.request.urlopen(virgo_url).read()
n=int(len(d)/4)
_arr=np.frombuffer(d,dtype=np.float32,count=n)

d=None

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    print("Finished loading")
    print('Load field in memory from virgo: {0:.2f} seconds\n'.format(t2-t1))
    print("shape == 3*(512**3): ",_arr.shape[0] == 3*(512**3))

[stdout:1] 
Finished loading
Load field in memory from virgo: 25.35 seconds

shape == 3*(512**3):  True
